In [2]:
import numpy as np, pandas as pd 
import matplotlib as plt
import seaborn as sns
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from math import sqrt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import streamlit as st
from sklearn.preprocessing import  LabelEncoder
from sklearn.linear_model import Lasso, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

trainingData = pd.read_csv('train.csv')
testingData = pd.read_csv('test.csv')

trainingData.shape
#trainingData.info()
trainingData.head()


,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [ ]:
trainingData.describe()


In [ ]:
print(trainingData.isna().sum().sort_values())

In [ ]:
trainingData.columns

In [ ]:
import matplotlib.pyplot as plt

numeric_data = trainingData.select_dtypes(include=[np.number])
# Calculate the correlation matrix
corr_matrix = numeric_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Draw the heatmap
# 'annot=True' to write the data value in each cell
# 'cmap' to choose the color palette
# 'fmt' to format the annotation if 'annot' is True
# 'linewidths' to set the width of the lines that will divide each cell
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5)

# Add a title to the heatmapx
plt.title('Correlation matrix of movie dataset')

# Show the plot
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size
sns.scatterplot(x='budget', y='revenue', data=trainingData)

# Adding a trend line to see the correlation more clearly
sns.regplot(x='budget', y='revenue', data=trainingData, scatter=False, color='red')

plt.title('Scatter plot of Budget vs. Revenue')  # Set the title of the plot
plt.xlabel('Budget')  # Set the x-axis label
plt.ylabel('Revenue')  # Set the y-axis label
plt.show()  # Display the plot


In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size
sns.scatterplot(x='popularity', y='revenue', data=trainingData)

# Adding a trend line to see the correlation more clearly
sns.regplot(x='popularity', y='revenue', data=trainingData, scatter=False, color='red')

plt.title('Scatter plot of Popularity vs. Revenue')  # Set the title of the plot
plt.xlabel('Popularity')  # Set the x-axis label
plt.ylabel('Revenue')  # Set the y-axis label
plt.show()  # Display the plot

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Remove 'id' column
trainingData = trainingData.drop('id', axis=1)

# Replace zero 'budget' and 'runtime' with the median (or mean)
for col in ['budget', 'runtime']:
    trainingData[col] = trainingData[col].replace(0, np.nan)  # Replace 0 with NaN
    median_value = trainingData[col].median()
    trainingData[col] = trainingData[col].fillna(median_value)  # Replace NaNs with the median

# Log transform 'revenue' to reduce the effect of outliers
trainingData['log_revenue'] = np.log1p(trainingData['revenue'])

# Feature scaling
scaler = StandardScaler()
numeric_cols = ['budget', 'popularity', 'runtime']  # list numeric columns
trainingData[numeric_cols] = scaler.fit_transform(trainingData[numeric_cols])


# Now, the 'trainingData' is adjusted and can be used for further modeling.
print(trainingData.isna().sum().sort_values())


In [ ]:
y = trainingData['revenue']
correlated_columns = ['budget', 'runtime', 'popularity']
X = trainingData.drop(['revenue', 'log_revenue'], axis=1)



In [ ]:
import json

def extract_id_from_collection(json_str):
    if pd.isna(json_str) or json_str == '[]':  # Check if the entry is NaN or an empty list
        return np.nan
    try:
        loaded_json = json.loads(json_str.replace("'", "\""))
        if loaded_json and 'id' in loaded_json[0]:
            return loaded_json[0]['id']
        else:
            return np.nan
    except json.JSONDecodeError:
        return np.nan

# Apply the function to the 'belongs_to_collection' column
X['collection_id'] = X['belongs_to_collection'].apply(extract_id_from_collection)

# Now, drop the original 'belongs_to_collection' column and any other non-numeric columns
X = X.drop('belongs_to_collection', axis=1)
X = X.select_dtypes(include=[np.number])

# Handle any NaN values created during the process
X = X.fillna(0)  # or you can choose to fill with median or mean

In [4]:
import matplotlib.pyplot as plt
df = pd.read_csv('train.csv')
X = df.drop(['revenue', 'id'], axis=1)  # Drop the target variable and ID
y = df['revenue']  # Target variable

#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of Train features: {}".format(X_train.shape))
print("Shape of Test features: {}".format(X_test.shape))
print("Shape of Train target: {}".format(y_train.shape))
print("Shape of Test target: {}".format(y_test.shape))


Shape of Train features: (2400, 21)
Shape of Test features: (600, 21)
Shape of Train target: (2400,)
Shape of Test target: (600,)


In [24]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

params = {"alpha":np.arange(0.00001, 10, 500)}
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')  # 'passthrough' or 'drop', depending on whether you have additional features not listed in numeric_features or categorical_features

# Fit the preprocessor on the full dataset X
X_processed = preprocessor.fit_transform(X)
#Number of Folds and adding the random state for replication
kf=KFold(n_splits=5,shuffle=True, random_state=42)

#Defining the model
lasso=Lasso()

#Defining the GridSearchCV
lasso_cv=GridSearchCV(lasso, param_grid=params, cv=kf)
lasso_cv.fit(X_processed, y)
print("Best Parameters: ", lasso_cv.best_params_)


Best Parameters:  {'alpha': 1e-05}


In [25]:
names=df.drop('revenue', axis=1).columns
print("Column Names: {}".format(names.values))

Column Names: ['id' 'belongs_to_collection' 'budget' 'genres' 'homepage' 'imdb_id'
 'original_language' 'original_title' 'overview' 'popularity'
 'poster_path' 'production_companies' 'production_countries'
 'release_date' 'runtime' 'spoken_languages' 'status' 'tagline' 'title'
 'Keywords' 'cast' 'crew']


In [26]:
lasso1 = Lasso(alpha=0.0001)
lasso1.fit(X_processed, y_train)

lasso1_coef = np.abs(lasso1.coef_)

# PLotting columns names and importance of columns
plt.bar(names, coefficients)
plt.xticks(rotation=90)
plt.grid()
plt.title("Feature Importance Based on Lasso")
plt.xlabel("Features")
plt.ylabel("Coefficient magnitude")
plt.ylim(0, 0.15)
plt.show()


ValueError: could not convert string to float: "[{'id': 230, 'name': 'The Godfather Collection', 'poster_path': '/7SJVjEDoo7xH62TS8stkqRk4Byo.jpg', 'backdrop_path': '/3WZTxpgscsmoUk81TuECXdFOD0R.jpg'}]"

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


#reg_all = LinearRegression()
#reg_all.fit(X_train, y_train)
#y_pred = reg_all.predict(X_test)

#xgb_reg = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.7, learning_rate = 0.2,
#                max_depth = 5, alpha = 1, n_estimators = 200)

#xgb_reg.fit(X_train, y_train)


# Define the RMSLE score function
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


# Create the RMSLE scorer
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

# Initialize the base model
rf = RandomForestRegressor(n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_depth=10, bootstrap=False)

rf.fit(X_train, y_train)
'''
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 300, stop = 3000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Execute the random search
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=100,  # You can choose different number of iterations
                               cv=3,  # You can change the number of folds here
                               verbose=2,
                               random_state=42,
                               n_jobs=-1,  # Use all available cores
                              scoring=rmsle_scorer)

# Fit the random search model on your training data (X_train, y_train)
rf_random.fit(X_train, np.log1p(y_train))  # using log1p transformation on y_train


# View the best parameters from fitting the random search
best_params = rf_random.best_params_
print(f"Best parameters found: {best_params}")

# Retrieve the best model
rf_optimized = rf_random.best_estimator_

# Predict on the test set using the optimized model
log_predictions_optimized = rf_optimized.predict(X_test)
predictions_optimized = np.expm1(log_predictions_optimized)  # Apply inverse log transformation

# Calculate the RMSLE on the test set
rmsle_optimized_score = rmsle(y_test, predictions_optimized)
'''
y_pred = rf.predict(X_test)
rf_score = rmsle(y_test, y_pred)
# Print the optimized RMSLE score
#print(f"The optimized RMSLE score is: {rmsle_optimized_score}")
print(f"The RMSLE score of the base model is: {rf_score}")



In [ ]:
#compute accuracy as proportion of rediction within given range
lower_bound = y_test * 0.8
upper_bound = y_test * 1.2

within_range = np.logical_and(y_pred >= lower_bound, y_pred <= upper_bound)

accuracy_within_range = np.mean(within_range)

print(f"accuracy within 80% of target variable: {accuracy_within_range:.2%} ")

In [ ]:
test_ids = testingData['id'].copy()

test_data = testingData.copy().drop('id', axis=1)

# Replace zero 'budget' and 'runtime' with the median (or mean)
for col in ['budget', 'runtime']:
    test_data[col] = test_data[col].replace(0, np.nan)  # Replace 0 with NaN
    median_value = trainingData[col].median()  # Use median from the training data
    test_data[col] = test_data[col].fillna(median_value)  # Replace NaNs with the median

# Feature scaling (use the same scaler object as you used for training data)
# Make sure to exclude 'log_revenue' from the list of columns to scale
numeric_cols = ['budget', 'popularity', 'runtime']  # 'log_revenue' should not be included
test_data[numeric_cols] = scaler.transform(test_data[numeric_cols])

# Apply the same JSON extraction to 'belongs_to_collection'
test_data['collection_id'] = test_data['belongs_to_collection'].apply(extract_id_from_collection)

# Now, drop the original 'belongs_to_collection' column and any other non-numeric columns
test_data = test_data.drop('belongs_to_collection', axis=1)
test_data = test_data.select_dtypes(include=[np.number])

# Handle any NaN values created during the process (use the same method as training data)
test_data = test_data.fillna(0)

# Predict on the preprocessed test_data
test_data_pred = rf.predict(test_data)

df = pd.DataFrame({"id": test_ids,
                   "revenue": test_data_pred})

df.to_csv('/DAT158/oblig2/submission.csv', index=False)